# Part 2: ADC Theory

In this task we will learn about how the built-in ADCs on a STM32 F446re Nucleo Board works, and how it is configured to exercise certain behaviours. There are three ADCs available to us, and all of them are [successive approximation register adcs (SAR)](https://en.wikipedia.org/wiki/Successive-approximation_ADC). Sampling with a SAR ADC is controlled by a clock signal, and happens in two stages:
1. The input voltage is fed to a capacitor, which can hold the charge while the A/D conversion occors. Once a certain number of clock cycles are completed, the input line is disconnected from the capacitor, leaving the current stored in  the capacitor with nowhere to go. The voltage over the capacitor will now remain constant.
2. A clever digital circuit connected to the capacitor uses something akin to [binary search](https://en.wikipedia.org/wiki/Binary_search) to "home in" on the precise capacitor voltage. The circuit will be able to do 1 bisection step (i.e. generating 1 bit of the ADC output value) per clock cycle, meaning a 12 bit A/D conversion requires at minimum 12 clock cycles. Then comes some overhead for clearing registers between conversions etc.

Below you can see a block diagram of the STM32.

![](Figures/ADC_block_diagram.png)


## Sampling frequency using countinuous successive approximation ADC in continuous mode

For simple systems using analog input without strict requirements to high sampling speed it is usually sufficient to keep the ADC in "single conversion mode", where each A/D conversion is initiated by a function call (such as `analogRead()` on an arduino). However, it is also possible to configure the ADC to begin a new A/D conversion immediately after completing the previous one. In this case, the sampling frequency will be determined by a number of factors which will vary depending on the type of ADC. For a Successive Approximation ADC such as the one in a STM32 F446RE, these factors are:

* **Peripheral Clock Signal**: A square wave clock signal generated by the processor, which controls the ADC (and a number of other components).
* **ADC Clock Prescaler**: An integer factor which determines the clock speed of the ADC relative to peripheral clock.
    * Example: If the peripheral clock is $84 \text{MHz}$, and the ADC clock prescaler is $8$, the ADC clock signal will be $\frac{84\text{ MHz}}{8} = 10.5 \text{MHz}$
* **Conversion time:** Determined by the ADC resolution. For the STM32 F446RE a 12-bit conversion (full resolution) requires $15$ clock cycles
* **Sample time:** Number of ADC clock cycles to wait while charging up the capacitor which will "hold" the sampled value once conversion begins.
    * Example: If the sample time is $3$ clock cycles, and the conversion time is $15$ clock cycles, then the total number of clock cycles per sample converted is $T = 3 + 15 = 18$.
 
## a)
Calculate the sampling frequency for an ADC with the following settings:
|Parameter|Value|
|---|---|
|Peripheral Clock| $$5.25 \text{ MHz}$$|
|ADC Clock Prescaler| $$8$$ |
|Sample Time | $$112\text{ cycles}$$ |
|Conversion Time | $$15 \text{ cycles}$$

In [13]:
Peripheral_Clock = 5.25*10**6
ADC_Clock_Prescaler = 8
T_s = 112
T_c = 15

System_Clock = Peripheral_Clock/ADC_Clock_Prescaler
Total_Cycles = System_Clock

Total_Number_Of_Samples = Total_Cycles / (T_s + T_c)

f_s  = Total_Number_Of_Samples

print(f_s)

5167.322834645669


In [14]:
from hashlib import sha1
assert sha1(str(round(float(f_s), 3)).encode('utf-8')+b'e585b').hexdigest() == '9609b91588c67a8c0d2c44434ec79ee986afeaae', 'Wrong answer :('
print('Correct answer :)')

Correct answer :)


## A short tutorial on register manipulation

A microprocessor usually configures a peripheral such as an ADC by making adjustments to one or more [*registers*](https://en.wikipedia.org/wiki/Processor_register). From a coder's perspective, a register is a specific part of the MCU's memory whose value is passed directly to the peripheral, and can be accessed using a certain global variable name which can be found in the MCU's reference manual. While many chip producers also provide software libraries which offer ways of configuring a peripheral without modifying a register directly, it is always useful to have a some knowledge of what's going on "under the hood" for those situations when the only option left is to hit the registers directly.

On a STM32, registers are singular 32-bit integer values, where each bit has a specific purpose. To adjust these, it is necessary to make use of [*bitwise operators*](https://en.wikipedia.org/wiki/Bitwise_operations_in_C). 

| Operator | Function | Example |
|---|---|---|
|`~`| Bitwise NOT | `~0b00001111` equals `0b11110000` | 
|`&`| Bitwise AND | `0b00001111 & 0b00111100` equals `0b00001100`|
|`\|`| Bitwise OR | `0b00001111 & 0b00111100` equals `0b00111111`|
|`^`| Bitwise XOR | `0b00001111 & 0b00111100` equals `0b00110011`|
|`>>`|Bitwise Right Shift| `0b11110000 >> 1` equals `0b01111000`|
|`<<`|Bitwise Left Shift | `0b00001111 << 1` equals `0b00011110`| 


According to the [STM32 F446RE referenc manual](Documents/dm00135183-stm32f446xx-advanced-arm-based-32-bit-mcus-stmicroelectronics.pdf) section 13.3, the ADC can be set to run in continuous mode by setting the bit in position 1 in *ADC control register 2* to `1`. Below is an example of how this can be achieved with bitwise operators:
```c
ADC1->CR2 |= (0x00000001 << 1) // Compound bitwise OR with all zeroes except bit nr. 1 from the right.
```
The inverse, i.e. setting bit nr. 1 to `0` and configuring the ADC to single conversion mode can be done as follows:
```c
ADC1->CR2 &= ~(0x00000001 << 1) // Compound bitwise AND with all ones except bit nr. 1 from the right.
```

## b) Register manipulation

Study the register table for `ADC sample time register 2` in section 13.3.5 of the reference manual. Suggest a code solution for configuring the sample time for ADC channel 2 to be 15 ADC clock cycles while leaving the rest of the register as is.

```C
// Write your C-code here. (it is a text cell, but highlighted as C-code)
ADC1->SMPR2  |= (0x1 << 3)
ADC1->SMPR2  &= ~(0x11 << 4)

```

PS. Python shares the same syntax for bitwise operations as `C`/`C++`. You can use python to do some trial and error if you want to experiment:

In [123]:
mock_register = 0b10101010  # Declare 8 lsb manually

mock_register |=  (0b1 << 3)
mock_register &=  ~(0b11 << 4)

print(f"{mock_register = :b}")  # Print register content after manipulation

mock_register = 10001010
